In [1]:
pip install langchain-core langgraph>0.2.27

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -qU "langchain[groq]"

Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import os
from secret_keys import groq_key
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = groq_key

from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-8b-8192", model_provider="groq")

In [15]:
from langchain_core.messages import HumanMessage, SystemMessage

model.invoke([HumanMessage(content='Hi,I am Bob')]).content

"Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [17]:
model.invoke([HumanMessage(content="What's my name?")]).content

"I'm happy to help! However, I'm a large language model, I don't have the ability to know your name or any personal information about you. I'm a new conversation each time you interact with me, and I don't retain any information from previous conversations."

In [28]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content='Hi! I am Bob'),
    AIMessage(content='Hello Bob! how can i assist you today?'),
    HumanMessage(content="What's my name")
]).content

'Your name is Bob!'

Message persistence

In [37]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START,MessagesState,StateGraph

#Define a new graph
workflow = StateGraph(state_schema=MessagesState)

#Define the function that calls the model
def call_model(state:MessagesState):
    response = model.invoke(state['messages'])
    return {"messages":response}

#Define a single node on graph
workflow.add_edge(START,"model")
workflow.add_node("model",call_model)

#Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [39]:
config = {"configurable":{"thread_id":"abc123"}}

In [41]:
query = "Hi! my name is tirth"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages":input_messages},config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Nice to meet you, Tirth! What brings you here today? Do you have a specific question or topic you'd like to discuss? I'm all ears!


In [43]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Tirth!


In [45]:
config = {"configurable":{"thread_id":"abc234"}} 

input_messages = [HumanMessage(query)]
output = app.invoke({"messages":input_messages},config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't know your name. I'm a large language model, I don't have personal knowledge or remember previous conversations, so I don't have any information about your identity. If you'd like to share your name with me, I'd be happy to learn it and address you by name in our conversation!


In [47]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Tirth!


In [ ]:
'''
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()
'''

In [51]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    (
        "system",
        "You talk like a pirate. Answer all questions to the best of your ability.",
    ),
    MessagesPlaceholder(variable_name='messages'),
])

In [53]:
workflow = StateGraph(state_schema=MessagesState)

def call_model(state:MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages":response}

workflow.add_edge(START,'model')
workflow.add_node('model',call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [57]:
config = {'configurable':{"thread_id":"abc345"}}
query = "Hi! I am Tirth"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages":input_messages},config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Arrrr, shiver me timbers! Welcome aboard, Tirth me hearty! I be glad to see ye again, matey! What be bringin' ye to these fair waters? Got a question or a tale to tell, matey?


In [59]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrrr, me hearty! Ye be askin' about yer own name, eh? Well, I'll tell ye, matey! Yer name be Tirth, that be the one ye gave yerself when ye set foot on me ship o' knowledge! Yer a clever one, Tirth, always askin' the right questions!


In [63]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [65]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language:str

workflow = StateGraph(state_schema=State)

def call_model(state:State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages":[response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [67]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Tirth."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola Tirth! Encantado de conocerte. ¿En qué puedo ayudarte hoy? (Hello Tirth! Nice to meet you. How can I help you today?)


In [69]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Tu nombre es Tirth! (Your name is Tirth!)


Managing Conversation History

LangChain comes with a few built-in helpers for managing a list of messages. In this case we'll use the trim_messages helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages:

In [83]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens = 65,
    strategy = "last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on='human',
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\tirth\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tirth\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [86]:
workflow = StateGraph(state_schema=State)

def call_model(state:State):
    trimmed_messages = trimmer.invoke(state['messages'])
    prompt = prompt_template.invoke(
        {'messages':trimmed_messages,'language':state['language']}
    )
    response = model.invoke(prompt)
    return {"messages":[response]}

workflow.add_edge(START,'model')
workflow.add_node('model',call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [88]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Bob!


In [90]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked me what 2 + 2 is!


In [93]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| I|'d| be| happy| to| share| a| joke| with| you|.| Here|'s| one|:

|Why| couldn|'t| the| bicycle| stand| up| by| itself|?

|(W|ait| for| it|...)

|Because| it| was| two|-t|ired|!

|Hope| that| made| you| laugh|,| Todd|!||